In [1]:
import pandas as pd
import numpy as np

In [2]:
def checkSomething(originalColumn, possibleDuplicates):
    for uniqueValue in originalColumn.unique():
        if len(possibleDuplicates.loc[originalColumn == uniqueValue].unique()) > 1:
            print(uniqueValue, ':', possibleDuplicates.loc[originalColumn == uniqueValue].unique())

In [3]:
df = pd.read_csv('line_score.csv')

In [4]:
df.columns

Index(['game_date_est', 'game_sequence', 'game_id', 'team_id_home',
       'team_abbreviation_home', 'team_city_name_home', 'team_nickname_home',
       'team_wins_losses_home', 'pts_qtr1_home', 'pts_qtr2_home',
       'pts_qtr3_home', 'pts_qtr4_home', 'pts_ot1_home', 'pts_ot2_home',
       'pts_ot3_home', 'pts_ot4_home', 'pts_ot5_home', 'pts_ot6_home',
       'pts_ot7_home', 'pts_ot8_home', 'pts_ot9_home', 'pts_ot10_home',
       'pts_home', 'team_id_away', 'team_abbreviation_away',
       'team_city_name_away', 'team_nickname_away', 'team_wins_losses_away',
       'pts_qtr1_away', 'pts_qtr2_away', 'pts_qtr3_away', 'pts_qtr4_away',
       'pts_ot1_away', 'pts_ot2_away', 'pts_ot3_away', 'pts_ot4_away',
       'pts_ot5_away', 'pts_ot6_away', 'pts_ot7_away', 'pts_ot8_away',
       'pts_ot9_away', 'pts_ot10_away', 'pts_away'],
      dtype='object')

In [5]:
df.head()

,game_date_est,game_sequence,game_id,team_id_home,team_abbreviation_home,team_city_name_home,team_nickname_home,team_wins_losses_home,pts_qtr1_home,pts_qtr2_home,...,pts_ot2_away,pts_ot3_away,pts_ot4_away,pts_ot5_away,pts_ot6_away,pts_ot7_away,pts_ot8_away,pts_ot9_away,pts_ot10_away,pts_away
0,1946-11-01 00:00:00,NaN,24600001,1610610035,HUS,Toronto,Huskies,-,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,68
1,1946-11-02 00:00:00,NaN,24600003,1610610034,BOM,St. Louis,Bombers,-,16.0,16.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51
2,1946-11-02 00:00:00,NaN,24600002,1610612738,BOS,Boston,Celtics,-,10.0,16.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59
3,1946-11-02 00:00:00,NaN,24600004,1610610025,CHS,Chicago,Stags,-,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47
4,1946-11-02 00:00:00,NaN,24600005,1610610036,WAS,Washington,Capitols,-,21.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33


## Normalize Team Information by Abbreviation

### Eliminate Duplicate City Names

In [6]:
checkSomething(df['team_abbreviation_home'], df['team_city_name_home']) 

KCK : ['Kansas City-Omaha' 'Kansas City']
LAC : ['Los Angeles' 'LA']
NOK : ['New Orleans' 'New Orleans/Oklahoma City']


In [7]:
df['team_nickname_home'].loc[df['team_abbreviation_home'] == 'NOK'].unique()

array(['Hornets'], dtype=object)

In [8]:
df = df.replace(['Kansas City-Omaha', 'LA'], ['Kansas City' , 'Los Angeles'])

df.loc[df['team_abbreviation_home'] == 'NOK', 'team_city_name_home'] = 'New Orleans/Oklahoma City'
df.loc[df['team_abbreviation_away'] == 'NOK', 'team_city_name_away'] = 'New Orleans/Oklahoma City'

In [9]:
checkSomething(df['team_abbreviation_home'], df['team_city_name_home'])
checkSomething(df['team_abbreviation_away'], df['team_city_name_away'])

### Eliminate Duplicate IDs

In [10]:
checkSomething(df['team_abbreviation_home'], df['team_id_home'])

WAS : [1610610036 1610612764]


In [11]:
print(df['team_abbreviation_home'].loc[df['team_id_home'] == 1610610036].unique())
print(df['team_abbreviation_home'].loc[df['team_id_home'] == 1610612764].unique())

['WAS']
['CHP' 'CHZ' 'BLT' 'CAP' 'WAS']


In [12]:
df.loc[df['team_abbreviation_home'] == 'WAS', 'team_id_home'] = 1610610036
df.loc[df['team_abbreviation_away'] == 'WAS', 'team_id_away'] = 1610610036

In [13]:
checkSomething(df['team_abbreviation_home'], df['team_id_home'])
checkSomething(df['team_abbreviation_away'], df['team_id_away'])

In [14]:
checkSomething(df['team_abbreviation_home'], df['team_city_name_home'])
checkSomething(df['team_abbreviation_away'], df['team_city_name_away'])

## Split Team Information with Abbreviation as the Primary Key

In [15]:
dfTeamInfo = df[['team_abbreviation_home', 'team_city_name_home', 'team_id_home']]
dfTeamInfo = dfTeamInfo.rename(columns = {'team_abbreviation_home': 'Abbreviation', 'team_city_name_home': 'City Name', 'team_id_home': 'ID'})
dfTeamInfo.head()

,Abbreviation,City Name,ID
0,HUS,Toronto,1610610035
1,BOM,St. Louis,1610610034
2,BOS,Boston,1610612738
3,CHS,Chicago,1610610025
4,WAS,Washington,1610610036


In [16]:
dfTeamInfo = dfTeamInfo.drop_duplicates(subset='Abbreviation', keep="first").reset_index(drop = True)
dfTeamInfo

,Abbreviation,City Name,ID
0,HUS,Toronto,1610610035
1,BOM,St. Louis,1610610034
2,BOS,Boston,1610612738
3,CHS,Chicago,1610610025
4,WAS,Washington,1610610036
...,...,...,...
70,NOK,New Orleans/Oklahoma City,1610612740
71,PHO,Phoenix,1610612756
72,OKC,Oklahoma City,1610612760
73,BKN,Brooklyn,1610612751
